In [ ]:
!pip install --upgrade onnx onnxscript onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.2/700.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Create a cache directory to store pretrained model.
import os
cache_dir = os.path.join(".", "cache_models")
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, AutoModel
import torch

model_name_or_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
config = AutoConfig.from_pretrained(model_name_or_path, cache_dir=cache_dir)
torch_model = AutoModelForCausalLM.from_pretrained(model_name_or_path, config=config, cache_dir=cache_dir, trust_remote_code=True)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
torch.save(torch_model, "/content/model.pth")

In [ ]:
device = torch.device('cpu')
torch_model.eval().to(device)

num_attention_heads = torch_model.config.num_attention_heads
hidden_size = torch_model.config.hidden_size
num_layer = torch_model.config.num_hidden_layers

In [ ]:
print(torch_model.config)

Qwen2Config {
  "_attn_implementation_autoset": true,
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 131072,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.50.2",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}



In [ ]:
from transformers.models.qwen2.modeling_qwen2 import DynamicCache

EXAMPLE_Text = ["The result of 2+2 is", "Give me some party ideas for a group of 5 people"]

def get_tokenizer(model_name_or_path, cache_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, cache_dir=cache_dir)
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer


def get_example_inputs(prompt_text=EXAMPLE_Text):
    tokenizer = get_tokenizer(model_name_or_path, cache_dir)
    encodings_dict = tokenizer.batch_encode_plus(prompt_text, padding=True)

    input_ids = torch.tensor(encodings_dict["input_ids"], dtype=torch.int32)
    attention_mask = torch.tensor(encodings_dict["attention_mask"], dtype=torch.int32)
    position_ids = attention_mask.long().cumsum(-1) - 1
    position_ids.masked_fill_(position_ids < 0, 0)
    position_ids = position_ids.to(torch.int32)

    # Empty Past State for generating first word
    empty_past = []
    batch_size = input_ids.size(0)
    sequence_length = input_ids.size(1)
    past_shape = [2, batch_size, num_attention_heads, 0, hidden_size // num_attention_heads]
    for i in range(num_layer):
        empty_past.append(torch.empty(past_shape).type(torch.float32).to(device))

    return input_ids, attention_mask, position_ids, empty_past



input_ids, attention_mask, position_ids, empty_past = get_example_inputs()
print("input_ids", input_ids)
print("attention_mask", attention_mask)
print("position_ids", position_ids)
print("empty_past", empty_past)

input_ids tensor([[151643, 151643, 151643, 151643, 151646,    785,   1102,    315,    220,
             17,     10,     17,    374],
        [151646,  35127,    752,   1045,   4614,   6708,    369,    264,   1874,
            315,    220,     20,   1251]], dtype=torch.int32)
attention_mask tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.int32)
position_ids tensor([[ 0,  0,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]],
       dtype=torch.int32)
empty_past [tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([], size=(2, 2, 12, 0, 128)), tensor([

In [ ]:
!huggingface-cli download onnxruntime/DeepSeek-R1-Distill-ONNX --include deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/* --local-dir .

Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]Downloading 'deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/special_tokens_map.json' to '.cache/huggingface/download/deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/ahkChHUJFxEmOdq5GDFEmerRzCY=.59cda48bbe8bab9d61ffb410e6e3c07b6d98bff73cee7c88ff8b51f95f21ab1c.incomplete'

model.onnx:   0% 0.00/197k [00:00<?, ?B/s]

tokenizer.json:   0% 0.00/11.4M [00:00<?, ?B/s]


tokenizer_config.json: 100% 6.75k/6.75k [00:00<00:00, 32.9MB/s]
Download complete. Moving file to deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/tokenizer_config.json
model.onnx: 100% 197k/197k [00:00<00:00, 5.89MB/s]
Download complete. Moving file to deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/model.onnx

special_tokens_map.json: 100% 485/485 [00:00<00:00, 2.83MB/s]
Download complete. Moving file to deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/cpu-int4-rtn

# Inference using onnxruntime

In [ ]:
import onnxruntime_genai as og

model = og.Model('/content/deepseek-r1-distill-qwen-1.5B/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

# Set the max length to something sensible by default,
# since otherwise it will be set to the entire context length
search_options = {}
search_options['max_length'] = 2048

chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

text = input("Input: ")
if not text:
   print("Error, input cannot be empty")
   exit

prompt = f'{chat_template.format(input=text)}'

input_tokens = tokenizer.encode(prompt)

params = og.GeneratorParams(model)
params.set_search_options(**search_options)

generator = og.Generator(model, params)
generator.append_tokens(input_tokens)

print("Output: ", end='', flush=True)

try:
   while not generator.is_done():
     generator.generate_next_token()

     new_token = generator.get_next_tokens()[0]
     print(tokenizer_stream.decode(new_token), end='', flush=True)
except KeyboardInterrupt:
    print("  --control+c pressed, aborting generation--")

print()
del generator

Input: What is 2+2?
Output: The answer is 4.
</think>

**Solution:**

To find the sum of 2 and 2, follow these steps:

1. **Add the numbers:**
   \[
   2 + 2 = 4
   \]

2. **Final Answer:**
   \[
   \boxed{4}
   \]


In [ ]:
import torch

def test_generation(tokenizer, input_text, ort_session=None, num_tokens_to_produce=30):
    assert len(input_text) == 1  # This function requires batch_size == 1
    use_onnxruntime = ort_session is not None
    print("Text generation using", "OnnxRuntime" if use_onnxruntime else "PyTorch", "...")
    eos_token_id = tokenizer.eos_token_id

    input_ids, attention_mask, position_ids, past = get_example_inputs(input_text)
    batch_size = input_ids.size(0)

    has_eos = torch.zeros(batch_size, dtype=torch.bool)

    all_token_ids = input_ids.clone()

    # Initialize past_key_values correctly based on your model's requirements
    empty_past = DynamicCache()

    for step in range(num_tokens_to_produce):
        if ort_session is not None:
            outputs = session.run(None, ort_inputs)
        else:
            outputs = torch_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_values=empty_past
            )

        next_token_logits = outputs[0][:, -1, :]
        next_tokens = torch.argmax(next_token_logits, dim=-1)

        has_eos |= next_tokens == eos_token_id
        tokens_to_add = next_tokens.masked_fill(has_eos, eos_token_id)
        all_token_ids = torch.cat([all_token_ids, tokens_to_add.unsqueeze(-1)], dim=-1)

        # Update inputs and past_key_values for the next step
        input_ids = tokens_to_add.clone().detach().reshape([batch_size, 1]).to(device)
        position_ids = (position_ids[:, -1] + 1).reshape(batch_size, 1)
        attention_mask = torch.cat([attention_mask, torch.ones([batch_size, 1]).type_as(attention_mask)], dim=1).to(device)

        empty_past = outputs[1] if not use_onnxruntime else [
            torch.from_numpy(outputs[i + 1]).to(device) if isinstance(outputs[i + 1], np.ndarray) else outputs[i + 1].clone().detach().to(device)
            for i in range(num_layer)
        ]

        if torch.all(has_eos):
            break

    for i, output in enumerate(all_token_ids):
        print("------------")
        print(tokenizer.decode(output.tolist(), skip_special_tokens=True))


In [ ]:
tokenizer = get_tokenizer(model_name_or_path, cache_dir)
input_text = EXAMPLE_Text[:1]
test_generation(tokenizer, input_text)

Text generation using PyTorch ...
------------
The result of 2+2 is 4. The result of 2+2+2 is 6. The result of 2+2+2+2 is 8
